In [2]:
import pandas as pd
import numpy as np
import os as os
#import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import classification_report

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
df = pd.read_csv('sentimento - base3.csv')

In [4]:
df.head()

id_issue    mes_chat SENTIMENTO     TIPO_MSG  \
0  15388962  2023-04-01     neutro       duvida   
1  17497395  2023-07-01     neutro       duvida   
2  15213047  2023-04-01     neutro  solicitacao   
3  17117301  2023-06-01     neutro     problema   
4  17345996  2023-07-01   negativo     problema   

                              hierarquia_4  \
0                        Vencimento fatura   
1        Dúvidas sobre pagamento de fatura   
2  Não reconhecimento de compra no crédito   
3                   Parcelamento de fatura   
4                              Dúvidas PIX   

                         ds_cluster_sociodemographic  \
0  0 - Mulher, Casada, 25 anos ou mais, cidade Pe...   
1  11 - Homem, Solteiro, Até 24 anos, cidade Pequ...   
2           17 - Mulher, 25 a 40 anos, Sudeste e Sul   
3  8 - Mulher, Solteira, Até 24 anos, cidade Pequ...   
4  10 - Mulher, Solteira, Até 24 anos, cidade Peq...   

                                          ds_message  flag_negativo_indecx  
0  ['Que dia minha fatura fecha?','Ainda tenho dú...                     0  
1  ['Quero pagar os 100 reais que eu tirei','Mais...                     0  
2  ['Apareceu uma compra que eu não fiz','Dúvida ...                     0  
3  ['No meu limite está escrito que está utilizad...                     0  
4  ['Pix','E merda o will nunca teve essas coisas...                     0

In [5]:
len(df)

14906

In [6]:
exemplo_base = df[df['SENTIMENTO'].notnull()]
len(exemplo_base)

645

In [7]:
stopwords_will= ['da', 'em', 'você', 'de', 'ao', 'os', 'que', 'e', 'eu', 'o', 'estou', 'ele','ela','pra', 'pro', 'entao', 'para', 'voce',
                 'pra', 'tá', 'né', 'ah', 'aí', 'dá', 'ó', 'mim', 'então', 'aqui', 'tô', 'pro',
            'seu', 'sua', 'por', 'sobre', 'mais', 'uma', 'um','como', 'meu', 'com', 'outro','outra','das','dos','foi', 'fiz','pelo','tem','mas',
            'este', 'esse','ta', 'to', 'ai', 'la', 'lo', 'lhe', 'ne','pra', 'tá', 'né', 'ah', 'aí', 'dá', 'ó', 'mim', 'então', 'aqui', 'tô', 'pro', 'isso']

In [8]:
exemplo_base['SENTIMENTO_label'] = np.where(exemplo_base['SENTIMENTO'] == "negativo", 0, 
                                            np.where(exemplo_base['SENTIMENTO'] == "neutro", 1, 2 ))

/tmp/ipykernel_69/2720231541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exemplo_base['SENTIMENTO_label'] = np.where(exemplo_base['SENTIMENTO'] == "negativo", 0,


In [9]:
exemplo_base['SENTIMENTO_label'].value_counts()

0    265
1    258
2    122
Name: SENTIMENTO_label, dtype: int64

In [10]:
!pip install optuna

  Obtaining dependency information for optuna from https://files.pythonhosted.org/packages/69/60/87a06ef66b34cbe2f2eb0ab66f003664404a7f40c21403a69fad7e28a82b/optuna-3.3.0-py3-none-any.whl.metadata
  Using cached optuna-3.3.0-py3-none-any.whl.metadata (17 kB)
  Obtaining dependency information for alembic>=1.5.0 from https://files.pythonhosted.org/packages/a2/8b/46919127496036c8e990b2b236454a0d8655fd46e1df2fd35610a9cbc842/alembic-1.12.0-py3-none-any.whl.metadata
  Using cached alembic-1.12.0-py3-none-any.whl.metadata (7.2 kB)
  Obtaining dependency information for cmaes>=0.10.0 from https://files.pythonhosted.org/packages/f7/46/7d9544d453346f6c0c405916c95fdb653491ea2e9976cabb810ba2fe8cd4/cmaes-0.10.0-py3-none-any.whl.metadata
  Using cached cmaes-0.10.0-py3-none-any.whl.metadata (19 kB)
  Using cached colorlog-6.7.0-py2.py3-none-any.whl (11 kB)
  Using cached Mako-1.2.4-py3-none-any.whl (78 kB)
Using cached optuna-3.3.0-py3-none-any.whl (404 kB)
Using cached alembic-1.12.0-py3-none-any.

In [11]:
import optuna
from sklearn.neural_network import MLPClassifier

vetorizador = TfidfVectorizer()
X = vetorizador.fit_transform(exemplo_base['ds_message'])
y = exemplo_base.SENTIMENTO_label


x_train, x_test, y_train, y_test = train_test_split(X
                                                    , exemplo_base.SENTIMENTO_label
                                                    , random_state=42, test_size=0.20)

In [ ]:
def objective(trial):
    hl = trial.suggest_int('hidden_layer_sizes', 90, 150, step=10)
    af = trial.suggest_categorical('activation', ['identity', 'tanh', 'relu'])
    alpha = trial.suggest_float("alpha", 0, 10)
    
    clf = MLPClassifier(solver='lbfgs', alpha=alpha, hidden_layer_sizes= hl, activation = af, warm_start = True)
    clf.fit(x_train, y_train)
    y_predicted = clf.predict(x_test)
    return metrics.cohen_kappa_score(y_test, y_predicted)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2023-10-10 17:50:22,110] A new study created in memory with name: no-name-99048bc9-493b-42f9-b4ff-e842cd81c1df
[I 2023-10-10 17:50:40,599] Trial 0 finished with value: 0.5869396468076848 and parameters: {'hidden_layer_sizes': 150, 'activation': 'relu', 'alpha': 1.1828640699231407}. Best is trial 0 with value: 0.5869396468076848.
[I 2023-10-10 17:50:47,657] Trial 1 finished with value: 0.4942663922375772 and parameters: {'hidden_layer_sizes': 90, 'activation': 'identity', 'alpha': 6.784981765938483}. Best is trial 0 with value: 0.5869396468076848.
[I 2023-10-10 17:51:03,697] Trial 2 finished with value: 0.5211487740548989 and parameters: {'hidden_layer_sizes': 150, 'activation': 'relu', 'alpha': 3.533196171891253}. Best is trial 0 with value: 0.5869396468076848.
[I 2023-10-10 17:51:20,837] Trial 3 finished with value: 0.4942663922375772 and parameters: {'hidden_layer_sizes': 100, 'activation': 'tanh', 'alpha': 6.618312354659053}. Best is trial 0 with value: 0.5869396468076848.
[I 202

In [18]:
study.best_params

{'hidden_layer_sizes': 130, 'activation': 'relu', 'alpha': 0.18641747634500205}

In [19]:
print(study.best_trial.value)

0.6149624494511843


In [20]:
model = MLPClassifier(**study.best_params, solver='lbfgs')
model.fit(x_train, y_train)

MLPClassifier(alpha=0.18641747634500205, hidden_layer_sizes=130, solver='lbfgs')

In [21]:
y_predicted = model.predict(x_test)
pd.crosstab(y_predicted, y_test)

SENTIMENTO_label   0   1   2
row_0                       
0                 44   4   6
1                 12  41   5
2                  2   2  13

In [22]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.81      0.76      0.79        58
           1       0.71      0.87      0.78        47
           2       0.76      0.54      0.63        24

    accuracy                           0.76       129
   macro avg       0.76      0.72      0.73       129
weighted avg       0.77      0.76      0.76       129



In [23]:
def objective(trial):
    
    params = {
        'learning_rate_init': trial.suggest_float('learning_rate_init', 0.0001, 0.1, step=0.005),
        'first_layer_neurons': trial.suggest_int('first_layer_neurons', 10, 200, step=50),
        'second_layer_neurons': trial.suggest_int('second_layer_neurons', 10, 100, step=10),
        'activation': trial.suggest_categorical('activation', ['identity', 'tanh', 'relu']),
    }

    model = MLPClassifier(
        hidden_layer_sizes=(params['first_layer_neurons'], params['second_layer_neurons']),
        learning_rate_init=params['learning_rate_init'],
        activation=params['activation'],
        random_state=1,
        max_iter=100
        )
    
    model.fit(x_train, y_train)
    y_predicted = model.predict(x_test)
    return metrics.cohen_kappa_score(y_test, y_predicted)

In [24]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2023-10-10 18:31:54,450] A new study created in memory with name: no-name-7c68e708-cda6-4b79-b64f-a0c6d68ca5f6
/opt/conda/lib/python3.8/site-packages/optuna/distributions.py:685: UserWarning: The distribution is specified by [0.0001, 0.1] and step=0.005, but the range is not divisible by `step`. It will be replaced by [0.0001, 0.0951].
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/optuna/distributions.py:700: UserWarning: The distribution is specified by [10, 200] and step=50, but the range is not divisible by `step`. It will be replaced by [10, 160].
  warnings.warn(
[I 2023-10-10 18:31:54,757] Trial 0 finished with value: 0.42958761895607034 and parameters: {'learning_rate_init ': 0.07010000000000001, 'first_layer_neurons': 10, 'second_layer_neurons': 40, 'activation': 'relu'}. Best is trial 0 with value: 0.42958761895607034.
/opt/conda/lib/python3.8/site-packages/optuna/distributions.py:685: UserWarning: The distribution is specified by [0.0001, 0.1] and step=0.005, but

In [25]:
study.best_params

{'learning_rate_init ': 0.07010000000000001,
 'first_layer_neurons': 160,
 'second_layer_neurons': 30,
 'activation': 'tanh'}

In [26]:
print(study.best_trial.value)

0.6008508992457939


In [33]:
model = MLPClassifier(learning_rate_init = 0.0701, hidden_layer_sizes = (160, 30), activation = 'tanh')

#clf = MLPClassifier(solver='lbfgs', alpha=alpha, hidden_layer_sizes= hl, activation = af, warm_start = True)

model.fit(x_train, y_train)

MLPClassifier(activation='tanh', hidden_layer_sizes=(160, 30),
              learning_rate_init=0.0701)

In [34]:
y_predicted = model.predict(x_test)
pd.crosstab(y_predicted, y_test)

SENTIMENTO_label   0   1   2
row_0                       
0                 45   8  10
1                 12  35   5
2                  1   4   9

In [35]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.71      0.78      0.74        58
           1       0.67      0.74      0.71        47
           2       0.64      0.38      0.47        24

    accuracy                           0.69       129
   macro avg       0.68      0.63      0.64       129
weighted avg       0.69      0.69      0.68       129

